# Prefix Tuning과 Prompt Tuning 완전 정복

## 학습 목표
이 강의를 통해 다음을 배우게 됩니다:
1. **Prefix Tuning**과 **Prompt Tuning**의 기본 개념
2. 두 방법이 왜 필요한지, 언제 사용하는지
3. EXAONE-3.5-7.8B-Instruct 모델을 활용한 실제 구현
4. 한국어 데이터셋을 이용한 실습

---

## 목차
1. [Parameter-Efficient Fine-Tuning(PEFT) 소개](#1-parameter-efficient-fine-tuningpeft-소개)
2. [Prefix Tuning 이해하기](#2-prefix-tuning-이해하기)
3. [Prompt Tuning 이해하기](#3-prompt-tuning-이해하기)
4. [실습: EXAONE 모델로 PrefixTuningConfig와 PromptTuningConfig 예제](#4-실습-exaone-모델로-prefixtungconfig와-prompttuningconfig-예제)
5. [데이터 전처리 및 학습](#5-데이터-전처리-및-학습)
6. [모델 테스트 및 성능 평가](#6-모델-테스트-및-성능-평가)
7. [성능 비교 및 정리](#7-성능-비교-및-정리)
8. [실습 과제 및 다음 단계](#8-실습-과제-및-다음-단계)


## 1. Parameter-Efficient Fine-Tuning(PEFT) 소개

### 전통적인 Fine-Tuning의 문제점

기존의 **전체 모델 Fine-Tuning**은 다음과 같은 문제가 있습니다:

- **메모리 부족**: 대형 모델(7B, 13B 파라미터)의 모든 파라미터를 업데이트하려면 엄청난 GPU 메모리가 필요
- **저장 공간**: 각 태스크마다 전체 모델을 별도로 저장해야 함
- **학습 시간**: 모든 파라미터를 업데이트하므로 학습이 오래 걸림
- **비용**: 높은 하드웨어 요구사항으로 인한 높은 비용

### PEFT의 해결책

**Parameter-Efficient Fine-Tuning(PEFT)**는 이러한 문제를 해결합니다:

- **적은 파라미터만 학습**: 전체 모델의 1% 미만만 업데이트
- **메모리 효율성**: 기존 대비 훨씬 적은 GPU 메모리 사용
- **빠른 학습**: 적은 파라미터로 인한 빠른 학습 속도
- **저장 효율성**: 작은 크기의 어댑터만 저장하면 됨

### PEFT의 주요 방법들

1. **LoRA (Low-Rank Adaptation)**: 행렬 분해를 통한 효율적 학습
2. **Prefix Tuning**: 입력 앞에 학습 가능한 prefix 추가
3. **Prompt Tuning**: 임베딩 레벨에서 가상 토큰 추가
4. **AdaLoRA**: 적응적 LoRA
5. **IA3**: Infused Adapter by Inhibiting and Amplifying Inner Activations


## 2. Prefix Tuning 이해하기

### PrefixTuningConfig란?

**PrefixTuningConfig**는 Prefix Tuning 방법을 설정하는 클래스입니다. 입력 시퀀스 앞에 **학습 가능한 연속적인 prefix**를 추가하는 방법입니다.

### 핵심 아이디어

```
일반적인 입력: [입력 토큰들]
Prefix Tuning: [학습가능한 prefix] + [입력 토큰들]
```

### 왜 PrefixTuningConfig가 필요한가?

#### 1. 메모리 효율성
- 전체 모델: 7.8B 파라미터 (약 15GB)
- Prefix Tuning: 약 1M 파라미터 (약 4MB)
- **99.9% 메모리 절약**

#### 2. 멀티태스크 지원
```python
# 하나의 기본 모델 + 여러 prefix
기본_모델 + 번역_prefix = 번역 모델
기본_모델 + 요약_prefix = 요약 모델
기본_모델 + QA_prefix = QA 모델
```

#### 3. 빠른 학습과 배포
- 작은 prefix만 학습하므로 빠름
- prefix 파일만 교체하면 즉시 태스크 변경 가능

### PrefixTuningConfig 주요 파라미터

- `task_type`: 태스크 유형 (CAUSAL_LM 등)
- `num_virtual_tokens`: 가상 토큰 개수
- `token_dim`: 토큰 차원 크기
- `num_transformer_submodules`: 트랜스포머 서브모듈 수
- `num_attention_heads`: 어텐션 헤드 수
- `num_layers`: 레이어 수


## 3. Prompt Tuning 이해하기

### PromptTuningConfig란?

**PromptTuningConfig**는 입력 임베딩 레벨에서 **학습 가능한 가상 토큰(soft prompt)**을 추가하는 방법을 설정하는 클래스입니다.

### Prefix Tuning vs Prompt Tuning

| 구분 | Prefix Tuning | Prompt Tuning |
|------|---------------|---------------|
| **적용 위치** | 모든 Transformer 레이어 | 입력 임베딩 레이어만 |
| **파라미터 수** | 더 많음 (레이어별 prefix) | 더 적음 (임베딩만) |
| **성능** | 일반적으로 더 높음 | 적지만 효율적 |
| **메모리** | 상대적으로 더 사용 | 매우 적게 사용 |

### 핵심 아이디어

```
일반적인 입력: [토큰1, 토큰2, 토큰3, ...]
Prompt Tuning: [가상토큰1, 가상토큰2, ..., 가상토큰k] + [토큰1, 토큰2, 토큰3, ...]
```

### 왜 PromptTuningConfig가 필요한가?

#### 1. 극도의 효율성
- **최소한의 파라미터**: 보통 100~500개의 가상 토큰만 사용
- **최소한의 메모리**: 몇 KB 수준의 추가 메모리만 필요
- **빠른 학습**: 매우 적은 파라미터로 인한 초고속 학습

#### 2. 자연스러운 프롬프트 방식
```python
# 전통적인 하드 프롬프트
"다음 문장을 한국어로 번역하세요: Hello world"

# Prompt Tuning의 소프트 프롬프트
[학습된_가상토큰들] + "Hello world"
```

#### 3. 모델 크기별 효과
- **작은 모델 (< 1B)**: 효과가 제한적
- **중간 모델 (1B~10B)**: 좋은 성능
- **대형 모델 (> 10B)**: 전체 fine-tuning과 비슷한 성능

### PromptTuningConfig 주요 파라미터

- `task_type`: 태스크 유형
- `prompt_tuning_init`: 초기화 방법 (TEXT, RANDOM 등)
- `num_virtual_tokens`: 가상 토큰 개수
- `prompt_tuning_init_text`: 초기화에 사용할 텍스트
- `tokenizer_name_or_path`: 토크나이저 경로


## 4. 실습: EXAONE 모델로 PrefixTuningConfig와 PromptTuningConfig 예제

이제 EXAONE-3.5-7.8B-Instruct 모델을 사용하여 실제로 두 방법을 구현해보겠습니다.


In [ ]:
# 필요한 라이브러리 설치
%pip install -q transformers datasets peft accelerate torch bitsandbytes


In [ ]:
# 필요한 라이브러리 임포트
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PrefixTuningConfig, PromptTuningConfig, PromptTuningInit, TaskType, get_peft_model
from datasets import Dataset

print("라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")


In [ ]:
# EXAONE 모델과 토크나이저 로드
model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

print("EXAONE 모델 로드 중...")

# 4bit 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

print("모델과 토크나이저 로드 완료!")
print(f"모델 크기: {sum(p.numel() for p in model.parameters())/1e9:.1f}B 파라미터")


In [ ]:
# PrefixTuningConfig 설정 예제
print("=== PrefixTuningConfig 설정 ===")

prefix_config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM,        # 인과적 언어 모델링 태스크
    inference_mode=False,                # 학습 모드
    num_virtual_tokens=30,               # 가상 토큰 개수
    token_dim=4096,                      # EXAONE 모델의 히든 차원
    num_transformer_submodules=2,        # 어텐션과 MLP
    num_attention_heads=32,              # EXAONE 모델의 어텐션 헤드 수
    num_layers=32,                       # EXAONE 모델의 레이어 수
    encoder_hidden_size=4096,            # 인코더 히든 크기
)

print("PrefixTuningConfig 설정 완료!")
print(f"설정 내용:")
print(f"   - 태스크 타입: {prefix_config.task_type}")
print(f"   - 가상 토큰 수: {prefix_config.num_virtual_tokens}")
print(f"   - 토큰 차원: {prefix_config.token_dim}")
print(f"   - 어텐션 헤드: {prefix_config.num_attention_heads}")
print(f"   - 레이어 수: {prefix_config.num_layers}")


In [ ]:
# PromptTuningConfig 설정 예제
print("=== PromptTuningConfig 설정 ===")

prompt_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,                    # 인과적 언어 모델링 태스크
    prompt_tuning_init=PromptTuningInit.TEXT,        # 텍스트로 초기화
    num_virtual_tokens=20,                           # 가상 토큰 개수 (Prefix보다 적게)
    prompt_tuning_init_text="한국어로 정확하고 친절하게 답변하세요",  # 초기화 텍스트
    tokenizer_name_or_path=model_name,               # 토크나이저 경로
)

print("PromptTuningConfig 설정 완료!")
print(f"설정 내용:")
print(f"   - 태스크 타입: {prompt_config.task_type}")
print(f"   - 가상 토큰 수: {prompt_config.num_virtual_tokens}")
print(f"   - 초기화 방법: {prompt_config.prompt_tuning_init}")
print(f"   - 초기화 텍스트: {prompt_config.prompt_tuning_init_text}")

# 파라미터 수 비교
prefix_params = 30 * 4096 * 2 * 32  # num_virtual_tokens * token_dim * submodules * layers
prompt_params = 20 * 4096           # num_virtual_tokens * token_dim

print(f"\n파라미터 수 비교:")
print(f"   - Prefix Tuning: {prefix_params:,} 파라미터")
print(f"   - Prompt Tuning: {prompt_params:,} 파라미터")
print(f"   - 차이: {(prefix_params/prompt_params):.1f}배")


In [ ]:
# 한국어 데이터셋 준비
print("=== 한국어 학습 데이터 준비 ===")

# 간단한 한국어 질문-답변 데이터셋
korean_data = [
    {
        "input": "안녕하세요! 오늘 날씨가 어때요?",
        "output": "안녕하세요! 저는 AI이므로 실시간 날씨 정보를 확인할 수 없지만, 날씨 앱이나 웹사이트를 통해 확인하시는 것을 추천드립니다."
    },
    {
        "input": "한국의 수도는 어디인가요?",
        "output": "한국의 수도는 서울특별시입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지입니다."
    },
    {
        "input": "김치는 어떤 음식인가요?",
        "output": "김치는 한국의 전통 발효 음식으로, 주로 배추나 무를 고춧가루, 마늘, 생강 등의 양념과 함께 절여서 만듭니다. 건강에 좋은 유산균이 풍부합니다."
    },
    {
        "input": "인공지능이란 무엇인가요?",
        "output": "인공지능(AI)은 인간의 지능을 모방하거나 인간이 수행하는 지적 작업을 기계가 수행할 수 있도록 하는 기술입니다."
    },
    {
        "input": "파이썬 프로그래밍의 특징은?",
        "output": "파이썬은 간결하고 읽기 쉬운 문법을 가진 프로그래밍 언어입니다. 다양한 라이브러리가 풍부하여 웹 개발, 데이터 분석, 머신러닝 등에 활용됩니다."
    }
]

print(f"한국어 학습 데이터 준비 완료!")
print(f"데이터 개수: {len(korean_data)}개")
print(f"예시 데이터:")
for i, data in enumerate(korean_data[:2], 1):
    print(f"   {i}. 입력: {data['input'][:30]}...")
    print(f"      출력: {data['output'][:30]}...")


In [ ]:
# PEFT 모델 생성 및 비교
print("=== PEFT 모델 생성 ===")

# Prefix Tuning 모델 생성
prefix_model = get_peft_model(model, prefix_config)

# Prompt Tuning 모델 생성 (새로운 모델 인스턴스 필요)
model_copy = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
prompt_model = get_peft_model(model_copy, prompt_config)

# 파라미터 정보 출력
def print_model_info(model, name):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(f"\n{name} 모델 정보:")
    print(f"   - 전체 파라미터: {total_params:,}")
    print(f"   - 학습 가능 파라미터: {trainable_params:,}")
    print(f"   - 학습 비율: {100 * trainable_params / total_params:.4f}%")
    model.print_trainable_parameters()

print_model_info(prefix_model, "Prefix Tuning")
print_model_info(prompt_model, "Prompt Tuning")


## 5. 데이터 전처리 및 학습

이제 실제로 한국어 데이터를 사용하여 두 모델을 학습해보겠습니다.


In [ ]:
# 데이터 전처리 및 학습 준비
from transformers import TrainingArguments, Trainer
import torch

def preprocess_function(examples):
    """데이터를 모델 학습에 적합한 형태로 전처리"""
    
    # EXAONE 모델용 프롬프트 템플릿
    def format_prompt(input_text, output_text):
        return f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다. 한국어로 정확하고 친절하게 답변해 주세요.[|endofturn|]\n[|user|]{input_text}[|endofturn|]\n[|assistant|]{output_text}[|endofturn|]"
    
    # 입력과 출력 결합
    texts = []
    for inp, out in zip(examples["input"], examples["output"]):
        text = format_prompt(inp, out)
        texts.append(text)
    
    # 토크나이징
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    
    # 라벨 설정 (입력과 동일)
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    
    return model_inputs

# Dataset 객체 생성 및 전처리
print("=== 데이터 전처리 ===")

# 리스트를 딕셔너리 형태로 변환
inputs = [data["input"] for data in korean_data]
outputs = [data["output"] for data in korean_data]

dataset_dict = {
    "input": inputs,
    "output": outputs
}

# Dataset 생성
train_dataset = Dataset.from_dict(dataset_dict)

# 전처리 적용
processed_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

print("데이터 전처리 완료!")
print(f"처리된 데이터셋 크기: {len(processed_dataset)}")
print(f"첫 번째 샘플 토큰 수: {len(processed_dataset[0]['input_ids'])}")


In [ ]:
# Prefix Tuning 학습 실행
print("=== Prefix Tuning 학습 ===")

# 학습 인자 설정
prefix_training_args = TrainingArguments(
    output_dir="./prefix_tuning_results",
    num_train_epochs=3,              # 에폭 수
    per_device_train_batch_size=1,   # 배치 크기 (메모리에 맞게 조정)
    gradient_accumulation_steps=4,   # 그래디언트 누적
    warmup_steps=10,                 # 워밍업 스텝
    learning_rate=1e-4,              # 학습률
    logging_steps=1,                 # 로깅 간격
    save_steps=50,                   # 저장 간격
    evaluation_strategy="no",        # 평가 전략
    save_total_limit=2,              # 저장할 체크포인트 수
    load_best_model_at_end=False,    # 최선 모델 로드
    report_to=None,                  # 리포팅 비활성화
    remove_unused_columns=False,     # 사용하지 않는 컬럼 제거 안함
    dataloader_pin_memory=False,     # 메모리 핀 비활성화
)

# 트레이너 생성
prefix_trainer = Trainer(
    model=prefix_model,
    args=prefix_training_args,
    train_dataset=processed_dataset,
    tokenizer=tokenizer,
)

print("학습 설정 완료!")
print(f"학습 설정:")
print(f"   - 에폭 수: {prefix_training_args.num_train_epochs}")
print(f"   - 배치 크기: {prefix_training_args.per_device_train_batch_size}")
print(f"   - 학습률: {prefix_training_args.learning_rate}")
print(f"   - 그래디언트 누적: {prefix_training_args.gradient_accumulation_steps}")

# 학습 시작
print("\nPrefix Tuning 학습 시작!")
prefix_trainer.train()

print("Prefix Tuning 학습 완료!")

# 모델 저장
prefix_model.save_pretrained("./saved_prefix_tuning_model")
print("Prefix Tuning 모델 저장 완료: ./saved_prefix_tuning_model")


In [ ]:
# Prompt Tuning 학습 실행
print("=== Prompt Tuning 학습 ===")

# 학습 인자 설정 (Prompt Tuning용으로 조정)
prompt_training_args = TrainingArguments(
    output_dir="./prompt_tuning_results",
    num_train_epochs=5,              # Prompt Tuning은 더 많은 에폭 필요
    per_device_train_batch_size=2,   # 파라미터가 적어서 배치 크기 증가 가능
    gradient_accumulation_steps=2,   # 그래디언트 누적 감소
    warmup_steps=5,                  # 워밍업 스텝 감소
    learning_rate=1e-3,              # 학습률 증가 (더 적은 파라미터)
    logging_steps=1,                 # 로깅 간격
    save_steps=25,                   # 저장 간격
    evaluation_strategy="no",        # 평가 전략
    save_total_limit=2,              # 저장할 체크포인트 수
    load_best_model_at_end=False,    # 최선 모델 로드
    report_to=None,                  # 리포팅 비활성화
    remove_unused_columns=False,     # 사용하지 않는 컬럼 제거 안함
    dataloader_pin_memory=False,     # 메모리 핀 비활성화
)

# 트레이너 생성
prompt_trainer = Trainer(
    model=prompt_model,
    args=prompt_training_args,
    train_dataset=processed_dataset,
    tokenizer=tokenizer,
)

print("학습 설정 완료!")
print(f"학습 설정:")
print(f"   - 에폭 수: {prompt_training_args.num_train_epochs}")
print(f"   - 배치 크기: {prompt_training_args.per_device_train_batch_size}")
print(f"   - 학습률: {prompt_training_args.learning_rate}")
print(f"   - 그래디언트 누적: {prompt_training_args.gradient_accumulation_steps}")

# 학습 시작
print("\nPrompt Tuning 학습 시작!")
prompt_trainer.train()

print("Prompt Tuning 학습 완료!")

# 모델 저장
prompt_model.save_pretrained("./saved_prompt_tuning_model")
print("Prompt Tuning 모델 저장 완료: ./saved_prompt_tuning_model")


## 6. 모델 테스트 및 성능 평가

학습이 완료된 모델들의 성능을 테스트해보겠습니다.


In [ ]:
# 학습된 모델 테스트
print("=== 학습된 모델 테스트 ===")

def generate_response(model, tokenizer, prompt, max_length=200):
    """주어진 프롬프트에 대해 모델이 응답을 생성하는 함수"""
    
    # EXAONE 프롬프트 형식
    formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다. 한국어로 정확하고 친절하게 답변해 주세요.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
    
    # 토크나이징
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # 응답 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # assistant 부분만 추출
    if "[|assistant|]" in response:
        assistant_response = response.split("[|assistant|]")[-1].strip()
        return assistant_response
    else:
        return response

# 테스트 질문들
test_questions = [
    "머신러닝과 딥러닝의 차이점은 무엇인가요?",
    "한국의 전통 음식 중 가장 유명한 것은?",
    "파이썬에서 리스트와 튜플의 차이는?",
    "서울에서 부산까지 가는 가장 빠른 방법은?",
    "인공지능의 미래는 어떻게 될까요?"
]

print("학습된 Prefix Tuning과 Prompt Tuning 모델의 응답을 비교해보겠습니다.\n")

for i, question in enumerate(test_questions[:2], 1):  # 처음 2개 질문만 테스트
    print(f"질문 {i}: {question}")
    print("-" * 60)
    
    # Prefix Tuning 모델 응답
    print("Prefix Tuning 응답:")
    try:
        prefix_response = generate_response(prefix_model, tokenizer, question)
        print(prefix_response)
    except Exception as e:
        print(f"오류 발생: {e}")
    
    print("\nPrompt Tuning 응답:")
    try:
        prompt_response = generate_response(prompt_model, tokenizer, question)
        print(prompt_response)
    except Exception as e:
        print(f"오류 발생: {e}")
    
    print("\n" + "="*80 + "\n")

print("추론 함수 정의 및 테스트 완료!")


In [ ]:
# 학습 결과 분석
print("=== 학습 결과 분석 ===")

import os

def get_folder_size(folder_path):
    """폴더 크기를 계산하는 함수"""
    total_size = 0
    if os.path.exists(folder_path):
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total_size += os.path.getsize(filepath)
    return total_size

# 저장된 모델 크기 비교
try:
    prefix_size = get_folder_size("./saved_prefix_tuning_model")
    prompt_size = get_folder_size("./saved_prompt_tuning_model")
    
    print(f"저장된 모델 크기 비교:")
    print(f"   - Prefix Tuning: {prefix_size / (1024*1024):.1f} MB")
    print(f"   - Prompt Tuning: {prompt_size / (1024*1024):.1f} MB")
    if prompt_size > 0:
        print(f"   - 크기 차이: {prefix_size / prompt_size:.1f}배")
except Exception as e:
    print(f"파일 크기 계산 중 오류 발생: {e}")

# 학습 완료 요약
print(f"\n학습 완료 요약:")
print(f"   - Prefix Tuning: 3 에폭, 학습률 1e-4")
print(f"   - Prompt Tuning: 5 에폭, 학습률 1e-3")
print(f"   - 사용 데이터: {len(korean_data)}개 한국어 QA 샘플")
print(f"   - 모델: EXAONE-3.5-7.8B-Instruct (4bit 양자화)")

# 성능 개선을 위한 추천사항
print(f"\n성능 개선을 위한 추천사항:")
print(f"   1. 더 많은 한국어 데이터 추가 (현재: {len(korean_data)}개)")
print(f"   2. 에폭 수 증가 (특히 Prompt Tuning)")
print(f"   3. 학습률 스케줄링 적용")
print(f"   4. 검증 데이터셋을 통한 조기 종료")
print(f"   5. 다양한 프롬프트 템플릿 실험")

print("\n모든 학습이 완료되었습니다!")


## 7. 성능 비교 및 정리

### 학습 결과 요약

| 방법 | 학습 가능 파라미터 | 메모리 사용량 | 학습 시간 | 추천 사용 시나리오 |
|------|------------------|--------------|----------|-------------------|
| **Prefix Tuning** | ~8M (0.1%) | 중간 | 중간 | 복잡한 태스크, 높은 성능 필요 |
| **Prompt Tuning** | ~80K (0.001%) | 매우 적음 | 매우 빠름 | 간단한 태스크, 빠른 적응 필요 |
| **전체 Fine-Tuning** | 7.8B (100%) | 매우 많음 | 매우 오래 걸림 | 최고 성능, 충분한 리소스 |

### 각 방법의 장단점

#### Prefix Tuning
**장점:**
- 높은 성능 (전체 fine-tuning에 근접)
- 적당한 메모리 사용량
- 복잡한 태스크에 효과적

**단점:**
- Prompt Tuning보다 많은 파라미터
- 설정이 복잡함

#### Prompt Tuning  
**장점:**
- 극도로 적은 파라미터 (99.999% 절약)
- 매우 빠른 학습
- 간단한 설정
- 대형 모델에서 효과적

**단점:**
- 작은 모델에서는 제한적 성능
- 복잡한 태스크에서는 한계

### 실제 사용 시 고려사항

1. **모델 크기**: 대형 모델(>10B)일수록 Prompt Tuning이 효과적
2. **태스크 복잡도**: 복잡한 태스크는 Prefix Tuning 권장
3. **리소스 제약**: 메모리나 시간이 제한적이면 Prompt Tuning
4. **데이터 크기**: 적은 데이터에서는 두 방법 모두 효과적


## 8. 실습 과제 및 다음 단계

### 실습 과제

1. **하이퍼파라미터 조정 실험**
   - `num_virtual_tokens` 값을 10, 50, 100으로 변경해서 성능 비교
   - `learning_rate`를 다양하게 조정해서 최적값 찾기

2. **다른 데이터셋으로 실험**
   - 자신만의 한국어 질문-답변 데이터 준비
   - 특정 도메인(의료, 법률, 기술 등)에 특화된 데이터로 학습

3. **다른 PEFT 방법과 비교**
   - LoRA와 성능 비교
   - AdaLoRA 실험

### 다음 단계 학습 권장사항

1. **Advanced PEFT 기법들**
   - QLoRA (Quantized LoRA)
   - MultiModal PEFT
   - PEFT 조합 기법

2. **실제 배포 고려사항**
   - 모델 서빙 최적화
   - 실시간 추론 성능
   - 멀티태스크 어댑터 관리

3. **평가 및 분석**
   - BLEU, ROUGE 등 정량적 평가
   - 사용자 만족도 평가
   - 오류 분석

### 추가 리소스

- **PEFT 공식 문서**: https://huggingface.co/docs/peft
- **Transformers 라이브러리**: https://huggingface.co/docs/transformers
- **EXAONE 모델 페이지**: https://huggingface.co/LGAI-EXAONE

---

### 마무리

이 강의를 통해 **Prefix Tuning**과 **Prompt Tuning**의 기본 개념부터 실제 구현까지 모든 과정을 학습했습니다. 

핵심 요약:
- **적은 비용으로 큰 효과**: 전체 모델의 1% 미만 파라미터로 좋은 성능
- **빠른 적응**: 새로운 태스크에 빠르게 적응 가능
- **실용적 솔루션**: 리소스 제약 환경에서 실용적인 해결책

더 궁금한 점이 있다면 언제든 질문해 주세요!
